In [10]:
# CSV 파일을 SQLite 데이터베이스로 전환하는 함수 정의
import csv
import sqlite3
import re

def csv_to_sqlite(csv_filepath, db_filepath, table_name, drop_existing_table=False):
    """
    CSV 파일을 읽어 SQLite 데이터베이스로 전환합니다.

    Args:
        csv_filepath (str): 변환할 CSV 파일의 경로.
        db_filepath (str): 생성하거나 연결할 SQLite 데이터베이스 파일의 경로.
        table_name (str): 데이터를 삽입할 SQLite 테이블의 이름.
    """
    try:
        conn = sqlite3.connect(db_filepath)
        cursor = conn.cursor()

        # drop_existing_table이 True이면 기존 테이블 삭제
        if drop_existing_table:
            drop_table_sql = f"DROP TABLE IF EXISTS {table_name};"
            cursor.execute(drop_table_sql)
            print(f"기존 테이블 '{table_name}'이(가) 삭제되었습니다 (존재하는 경우).")
            conn.commit() # DROP TABLE 후에는 바로 커밋하여 반영

        with open(csv_filepath, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            headers = next(reader)  # 첫 번째 행을 헤더로 읽음

            # SQL 컬럼 이름으로 유효하게 변환 (특수 문자 제거 및 공백 대체)
            cleaned_headers = [re.sub(r'\W+', '', header).strip().lower() for header in headers]
            
            # 테이블 생성 쿼리 구성
            # 모든 컬럼을 기본적으로 TEXT 타입으로 지정하여 유연성을 높입니다.
            # 필요에 따라 INSERT 전에 데이터 타입 변환 로직을 추가할 수 있습니다.
            columns_def = ', '.join([f"{header} TEXT" for header in cleaned_headers])
            create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_def});"
            cursor.execute(create_table_sql)
            print(f"테이블 '{table_name}'이(가) 생성되었거나 이미 존재합니다.")

            # 데이터 삽입 쿼리 구성
            placeholders = ', '.join(['?' for _ in cleaned_headers])
            insert_sql = f"INSERT INTO {table_name} ({', '.join(cleaned_headers)}) VALUES ({placeholders});"

            # 데이터 삽입
            for row_num, row in enumerate(reader, start=2): # 첫 번째 행은 헤더이므로 2부터 시작
                if len(row) != len(cleaned_headers):
                    print(f"경고: {row_num}행의 컬럼 개수가 헤더와 일치하지 않습니다. 이 행은 건너뜁니다: {row}")
                    continue
                try:
                    cursor.execute(insert_sql, row)
                except sqlite3.Error as e:
                    print(f"오류: {row_num}행을 삽입하는 중 오류가 발생했습니다: {row}. 오류 메시지: {e}")
                    continue # 오류 발생 시 해당 행은 건너뛰고 다음 행으로 진행

            conn.commit()
        print(f"'{csv_filepath}' 파일의 데이터가 '{db_filepath}'의 '{table_name}' 테이블로 성공적으로 전환되었습니다.")

    except FileNotFoundError:
        print(f"오류: '{csv_filepath}' 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    except sqlite3.Error as e:
        print(f"SQLite 데이터베이스 오류: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
    finally:
        if 'conn' in locals() and conn:
            conn.close()

In [11]:
# CSV 파일을 SQLite 데이터베이스로 전환하는 함수 호출
csv_file = "../data/임베딩대상_202505261852.csv"
sqlite_db = "../.db/sqlite/my_database.db"
table = "PRODUCT"

csv_to_sqlite(csv_file, sqlite_db, table, True)

# 3. 데이터 확인 (선택 사항)
print("\n--- SQLite 데이터 확인 ---")
try:
    conn = sqlite3.connect(sqlite_db)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except sqlite3.Error as e:
    print(f"데이터 확인 중 오류 발생: {e}")
finally:
    if 'conn' in locals() and conn:
        conn.close()

기존 테이블 'PRODUCT'이(가) 삭제되었습니다 (존재하는 경우).
테이블 'PRODUCT'이(가) 생성되었거나 이미 존재합니다.
'../data/임베딩대상_202505261852.csv' 파일의 데이터가 '../.db/sqlite/my_database.db'의 'PRODUCT' 테이블로 성공적으로 전환되었습니다.

--- SQLite 데이터 확인 ---
('0021216860', '엔산 EL-09 이젤형 TV스탠드 티비거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '색상', '화이트 계열', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '176000', '176000', '0', '', '', '176000')
('0026595956', '엔산 무드조명형 모니터암 블룸스탠드 삼텐바이미 거치대', '정상상품', '엔산마운트', 'A/V 기타>TV 기타', '', '', 'TV·영상가전>TV>TV액세서리', 'N', '#TV거치대#TV브라켓#모니터거치대#모니터브라켓', '140000', '140000', '0', '', '', '140000')
('0037357623', '5년무상AS 이스트라 120Hz AI 맥스 127cm UHD 구글 5.0 스마트TV (이젤스탠드A타입-기사)', '정상상품', '이스트라', 'TV>UHD TV', '크기,종류,해상도,응답속도,기본 주사율,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,스마트기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,부가기능,e효율등급,출시년도,화면 타입,스피커 출력,음향효과,음향효과,지원단자', '50인치(127cm),LED,UHD(4K),6m/s,120Hz,구글플레이,넷플릭스,유튜브,크롬캐스트,구글어시스턴트,Disney+,블루투스,게임모드,플리커프리,HDMI CEC,HDMI 2.1 지원,HDMI eARC,크로마서브샘플링(4:4:4),USB재생(동영상·사진·음악),1등급,2025년,평면형,24W,

In [12]:
# 주어진 컬럼들을 바탕으로 ML 검색에 용이하도록 문맥에 맞게 병합

# 컬럼 인덱스 정의 (실제 CSV 파일의 컬럼 순서에 맞게 조정해야 합니다)
# 이 부분은 실제 데이터의 컬럼 순서와 일치해야 합니다.
COLUMN_INDICES = {
    "GOODS_NO": 0,
    "GOODS_NM": 1,
    "GOODS_STAT_SCT_NM": 2,
    "BRND_NM": 3,
    "ARTC_INFO": 4,
    "OPT_DISP_NM": 5,
    "OPT_VAL_DESC": 6,
    "CATEGORY_NMS": 7,
    "APPLIANCE_YN": 8,
    "SCH_KWD_NM": 9,
    "SALE_PRC": 10,
    "DSCNT_SALE_PRC": 11,
    "CARD_DC_RATE": 12,
    "CARD_DC_NAME": 13,
    "CARD_DC_NAME_LIST": 14,
    "MAX_BENEFIT_PRICE": 15
}

def generate_search_text_from_tuple(product_data_tuple):
    """
    주어진 상품 정보 튜플에서 검색에 최적화된 통합 텍스트 문자열을 생성합니다.
    머신러닝 기반의 의미론적 검색에 유리하도록 문맥을 구성합니다.

    Args:
        product_data_tuple (tuple): 상품 정보를 담고 있는 튜플.
                                  COLUMNS_INDICES에 정의된 순서와 일치해야 합니다.

    Returns:
        str: 검색 인덱싱을 위한 상품 정보를 결합한 문자열.
    """

    search_components = []

    # 튜플에서 데이터 추출 (COLUMN_INDICES를 활용하여 가독성을 높입니다)
    goods_nm = product_data_tuple[COLUMN_INDICES["GOODS_NM"]].strip()
    goods_stat_sct_nm = product_data_tuple[COLUMN_INDICES["GOODS_STAT_SCT_NM"]].strip()
    brnd_nm = product_data_tuple[COLUMN_INDICES["BRND_NM"]].strip()
    artc_info = product_data_tuple[COLUMN_INDICES["ARTC_INFO"]].strip()
    opt_disp_nm = product_data_tuple[COLUMN_INDICES["OPT_DISP_NM"]].strip()
    opt_val_desc = product_data_tuple[COLUMN_INDICES["OPT_VAL_DESC"]].strip()
    category_nms = product_data_tuple[COLUMN_INDICES["CATEGORY_NMS"]].strip()
    appliance_yn = product_data_tuple[COLUMN_INDICES["APPLIANCE_YN"]].strip()
    sch_kwd_nm = product_data_tuple[COLUMN_INDICES["SCH_KWD_NM"]].strip()
    
    # 숫자 값은 형 변환 시 에러 방지를 위해 get 대신 직접 접근 후 try-except 처리
    sale_prc_str = product_data_tuple[COLUMN_INDICES["SALE_PRC"]]
    dscnt_sale_prc_str = product_data_tuple[COLUMN_INDICES["DSCNT_SALE_PRC"]]
    card_dc_rate_str = product_data_tuple[COLUMN_INDICES["CARD_DC_RATE"]]
    max_benefit_price_str = product_data_tuple[COLUMN_INDICES["MAX_BENEFIT_PRICE"]]
    
    sale_prc = int(sale_prc_str) if sale_prc_str else None
    dscnt_sale_prc = int(dscnt_sale_prc_str) if dscnt_sale_prc_str else None
    card_dc_rate = int(card_dc_rate_str) if card_dc_rate_str else None
    max_benefit_price = int(max_benefit_price_str) if max_benefit_price_str else None

    card_dc_name = product_data_tuple[COLUMN_INDICES["CARD_DC_NAME"]].strip()


    # 1. 브랜드명 + 품목 정보 (카테고리/ARTC_INFO) + 상품명 조합
    # 상품의 핵심적인 정체성을 명확히 합니다.
    if brnd_nm and goods_nm:
        # ARTC_INFO에서 품목의 구체적인 타입을 추론하여 문맥에 더합니다.
        item_type = ''
        if artc_info:
            # '>' 기호로 구분된 품목 정보 중 가장 마지막 부분을 가져와 특정 품목 타입을 얻습니다.
            item_type_parts = [part.strip() for part in artc_info.split('>') if part.strip()]
            if item_type_parts:
                item_type = item_type_parts[-1]
                if '기타' in item_type: # '기타' 라는 모호한 품목이 포함되어 있는 경우
                    if category_nms: # 카테고리의 마지막 부분을 가져옵니다.
                        main_category_parts = [part.strip() for part in category_nms.split('>')]
                        if main_category_parts:
                            item_type = main_category_parts[0]


        # 품목 타입이 있고, 상품명에 이미 품목 타입이 포함되어 있지 않다면 추가합니다.
        if item_type and item_type not in goods_nm:
            search_components.append(f"{brnd_nm}의 {item_type}, {goods_nm}.")
        else: # 품목 타입이 없거나 상품명에 이미 포함되어 있다면 브랜드명과 상품명만 사용합니다.
            search_components.append(f"{brnd_nm}의 {goods_nm}.")
    elif goods_nm: # 브랜드명이 없는 경우 상품명만 사용합니다.
        search_components.append(f"{goods_nm}.")


    # 2. 제품의 주요 특성 (OPT_DISP_NM 및 OPT_VAL_DESC) 조합
    if opt_disp_nm and opt_val_desc:
        display_names = [name.strip() for name in opt_disp_nm.split(',') if name.strip()]
        values = [val.strip() for val in opt_val_desc.split(',') if val.strip()]

        # 각 특성 항목과 값을 '항목: 값' 형태로 결합합니다.
        characteristics = []
        for i in range(min(len(display_names), len(values))):
            # 비어있는 항목이나 '스마트기능'처럼 반복되는 용어는 건너뜁니다.
            # '스마트기능'이 반복되는 경우, 첫 번째 '스마트기능' 뒤에 모든 스마트기능 값을 나열합니다.
            if display_names[i] == '스마트기능':
                if i > 0 and display_names[i-1] == '스마트기능':
                    continue # 이미 처리되었으므로 건너뜁니다.
                
                # 모든 스마트기능 값을 모아서 처리
                all_smart_features = []
                for j in range(i, len(display_names)):
                    if display_names[j] == '스마트기능' and j < len(values):
                        all_smart_features.append(values[j])
                    else:
                        break # 스마트기능이 끝나면 반복 중단

                if all_smart_features:
                    characteristics.append(f"스마트기능: {', '.join(all_smart_features)}")
            else: # 일반적인 특성
                if display_names[i] and values[i]:
                    characteristics.append(f"{display_names[i]}: {values[i]}")

        if characteristics:
            search_components.append("주요 특징: " + ", ".join(characteristics) + ".")

    # 3. 카테고리명 추가
    if category_nms:
        # '|' 기호를 ', '로, '>' 기호를 ' > '로 바꾸어 카테고리 계층 및 다중 카테고리를 표현합니다.
        formatted_categories = category_nms.replace('|', ', ').replace('>', ' > ')
        search_components.append(f"카테고리: {formatted_categories}.")

    # 4. 상품 상태 추가
    if goods_stat_sct_nm and goods_stat_sct_nm != '정상상품': # '정상상품'이 아닌 경우에만 추가합니다.
        search_components.append(f"상품 상태: {goods_stat_sct_nm}.")

    # 5. 가격 및 할인 정보 추가
    price_info_components = []
    if sale_prc is not None:
        price_info_components.append(f"판매가 {sale_prc:,}원") # 천단위 콤마 추가
    if dscnt_sale_prc is not None and sale_prc is not None and dscnt_sale_prc < sale_prc:
        price_info_components.append(f"할인가 {dscnt_sale_prc:,}원") # 천단위 콤마 추가
    if card_dc_rate is not None and card_dc_rate > 0 and card_dc_name:
        if max_benefit_price is not None:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인되어 최대 {max_benefit_price:,}원") # 천단위 콤마 추가
        else:
            price_info_components.append(f"{card_dc_name} 카드 사용 시 {card_dc_rate}% 할인")

    if price_info_components:
        search_components.append("가격 정보: " + ", ".join(price_info_components) + ".")

    # 6. 가전제품 여부 추가
    if appliance_yn == 'Y':
        search_components.append("가전제품입니다.")
    else:
        search_components.append("가전제품이 아닙니다.")

    # 7. 해시태그 추가
    if sch_kwd_nm: 
        search_components.append(f"해시태그: {sch_kwd_nm}.")
    
    # 모든 구성 요소를 하나의 문자열로 결합하고 앞뒤 공백을 제거합니다.
    return " ".join(search_components).strip()

In [13]:


# 임베딩 텍스트 생성
for row in rows:
    search_text = generate_search_text_from_tuple(row)

search_text  

'LG전자의 LG 트롬 드럼 세탁기 24kg F24WDWP. 주요 특징: e효율등급: 1등급. 카테고리: 세탁기·건조기·의류관리기 > 드럼세탁기. 가격 정보: 판매가 1,056,300원, 할인가 961,230원. 가전제품입니다. 해시태그: #24kg#F24WDWP#LG#드럼#드럼세세탁기#세탁기#엘지세탁기#트롬.'